In [7]:
import gymnasium as gym
from collections import defaultdict
import random

In [36]:
policy = defaultdict(int)
state_action_value = defaultdict(float)
state_action_total_rewards = defaultdict(int)
num_visited = defaultdict(int)

env = gym.make('FrozenLake-v1', is_slippery = True)
observation, info = env.reset()

epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01
gamma = 1.0
for _ in range(100_000):
    observation, _ = env.reset()
    episode_over = False
    episodes = []
    while not episode_over:
        if random.random() > epsilon:
            action = policy[observation]
        else:
            action = env.action_space.sample()
        episodes.append((observation, action))
        observation, r, terminated, truncated, _ = env.step(action)

        if terminated or truncated:
            episode_over = True
            reward = r
    seen = set()
    for i in range(len(episodes) - 1, -1, -1):
        state_action = (episodes[i][0], episodes[i][1])
        #reward *= gamma
        if state_action not in seen:
            seen.add(state_action)
            state_action_total_rewards[state_action] += reward
            num_visited[state_action] += 1
            state_action_value[state_action] = state_action_total_rewards[state_action] / num_visited[state_action]
            best_action = max(range(env.action_space.n), key=lambda a: state_action_value[(episodes[i][0], a)])
            policy[episodes[i][0]] = best_action
    
    epsilon *= epsilon_decay
    epsilon = max(epsilon, min_epsilon)

env.close()

In [41]:
num_success = 0
for _ in range(1000):
    observation, _ = env.reset()
    episode_over = False

    while not episode_over:
        action = policy[observation]
        observation, reward, terminated, truncated, _ = env.step(action)
        
        if terminated or truncated:
            episode_over = True
            num_success += reward

print(f"Success rate over 100 episodes: {num_success/10}%")

Success rate over 100 episodes: 72.7%
